In [20]:
from pprint import pprint
import pandas as pd
df_nurse_record = pd.read_csv("TestData/NurseRecord_full.csv")

/var/folders/pj/6yc3mtz11t5bfkf7bljn52z40000gn/T/ipykernel_1433/3962164412.py:3: DtypeWarning: Columns (9,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nurse_record = pd.read_csv("TestData/NurseRecord_full.csv")


In [27]:
patient_id = 10216420

In [22]:
import json
def prettyjson(final_dict):
    print(json.dumps(final_dict, ensure_ascii=False, indent=4))

In [21]:
import json
def get_nursing_record(patient_id: int):
    patient_data = df_nurse_record[df_nurse_record['연구등록번호'] == patient_id]
    columns = ['간호중재_코드명', '간호활동_코드명', '간호속성코드_코드명', '속성', 'Duty_코드명', '시행일시']
    result = patient_data[columns].sort_values('시행일시')

    # 시행일시별로 그룹화
    final_dict = {}
    grouped = result.groupby('시행일시')

    for time_key, group in grouped:
        # 각 그룹의 데이터를 리스트로 변환
        time_data = []
        for index, row in group.iterrows():
            time_data.append({
                "간호중재(코드명)": row['간호중재_코드명'],
                "간호활동(코드명)": row['간호활동_코드명'], 
                "간호속성코드(코드명)": row['간호속성코드_코드명'],
                "속성": row['속성'],
                "Duty(코드명)": row['Duty_코드명'],
                "시행일시": row['시행일시']
            })
        
        final_dict[time_key] = time_data
    return final_dict

# print(json.dumps(final_dict, ensure_ascii=False, indent=4))

In [ ]:
DataConverter = {
    "admission_periods": [
        {
            "start": "??",
            "end": "!!",
            "id": "admission1"
        }
    ],
    "alarms": {},
    "nursing_records": get_nursing_record(patient_id),
    "waveforms": {}
}

In [ ]:
import pandas as pd
import os
from pathlib import Path

# NURSE 폴더 생성
output_dir = Path("NURSE")
output_dir.mkdir(exist_ok=True)

# 모든 연구등록번호 찾기
all_patient_ids = df_nurse_record['연구등록번호'].unique()
print(f"총 {len(all_patient_ids)}명의 환자 데이터를 처리합니다.")

columns = ['간호중재_코드명', '간호활동_코드명', '간호속성코드_코드명', '속성', 'Duty_코드명', '시행일시']

for patient_id in all_patient_ids:
    print(f"처리 중: 연구등록번호 {patient_id}")
    
    # 해당 환자의 데이터 필터링
    patient_data = df_nurse_record[df_nurse_record['연구등록번호'] == patient_id]
    result = patient_data[columns].sort_values('시행일시')
    
    # 데이터가 없으면 스킵
    if result.empty:
        print(f"  - 연구등록번호 {patient_id}: 데이터 없음")
        continue
    
    # 컬럼명을 괄호 버전으로 변경
    result_renamed = result.rename(columns={
        '간호중재_코드명': '간호중재(코드명)',
        '간호활동_코드명': '간호활동(코드명)',
        '간호속성코드_코드명': '간호속성코드(코드명)',
        'Duty_코드명': 'Duty(코드명)'
    })
    
    # CSV 파일로 저장 (평평한 구조)
    csv_filename = output_dir / f"{patient_id}.csv"
    result_renamed.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"  - 저장완료: {csv_filename} ({len(result)}개 기록)")

print(f"\n전체 처리 완료! {output_dir} 폴더에 저장되었습니다.")

# 처리 결과 요약
print("\n=== 처리 결과 요약 ===")
saved_files = list(output_dir.glob("*.csv"))
print(f"저장된 파일 수: {len(saved_files)}")
print(f"총 환자 수: {len(all_patient_ids)}")
if saved_files:
    print(f"예시 파일들: {[f.name for f in saved_files[:5]]}")

총 545명의 환자 데이터를 처리합니다.
처리 중: 연구등록번호 11729736
  - 저장완료: NURSE/11729736.json
처리 중: 연구등록번호 11719538
  - 저장완료: NURSE/11719538.json
처리 중: 연구등록번호 11719074
  - 저장완료: NURSE/11719074.json
처리 중: 연구등록번호 11716267
  - 저장완료: NURSE/11716267.json
처리 중: 연구등록번호 11713362
  - 저장완료: NURSE/11713362.json
처리 중: 연구등록번호 11719073
  - 저장완료: NURSE/11719073.json
처리 중: 연구등록번호 11711062
  - 저장완료: NURSE/11711062.json
처리 중: 연구등록번호 11708731
  - 저장완료: NURSE/11708731.json
처리 중: 연구등록번호 11701700
  - 저장완료: NURSE/11701700.json
처리 중: 연구등록번호 11698881
  - 저장완료: NURSE/11698881.json
처리 중: 연구등록번호 11693779
  - 저장완료: NURSE/11693779.json
처리 중: 연구등록번호 11688056
  - 저장완료: NURSE/11688056.json
처리 중: 연구등록번호 11691445
  - 저장완료: NURSE/11691445.json
처리 중: 연구등록번호 11687877
  - 저장완료: NURSE/11687877.json
처리 중: 연구등록번호 11685481
  - 저장완료: NURSE/11685481.json
처리 중: 연구등록번호 11679744
  - 저장완료: NURSE/11679744.json
처리 중: 연구등록번호 11678265
  - 저장완료: NURSE/11678265.json
처리 중: 연구등록번호 11680244
  - 저장완료: NURSE/11680244.json
처리 중: 연구등록번호 11684395
  - 저장완료: NURSE/116

In [ ]:
import json



In [26]:
def get_admission_periods(patient_id):
    r = json.loads(open(f'TestData/{patient_id}.json', 'r', encoding='utf-8').read())
    admission_periods = []
    for idx, i in enumerate(sorted(list(r['DatSeq'].keys()), key=int)):
        alarms = list(r['DatSeq'][i].keys())
        admission_periods.append({
            "start": alarms[0].split(' ')[0],
            "end": alarms[-1].split(' ')[0],
            "id": f"admission{idx+1}"
        })
    prettyjson(admission_periods)

In [28]:
get_admission_periods(patient_id)

[
    {
        "start": "2024-01-09",
        "end": "2024-01-10",
        "id": "admission1"
    },
    {
        "start": "2024-01-10",
        "end": "2024-01-11",
        "id": "admission2"
    }
]


In [8]:
r['DatSeq']

KeyError: 'DatSeq'

In [ ]:
r['DatSeq']['2']['2024-02-28 23:10:35.923'][0].keys()

dict_keys(['TimeStamp', 'PatSeq', 'SequenceNumber', 'AlertId', 'Source', 'Label', 'Severity', 'Kind', 'AnnounceTime', 'AlertTime', 'II_wave', 'II_time_diff_sec', 'ABP_wave', 'ABP_time_diff_sec', 'Pleth_wave', 'Pleth_time_diff_sec', 'Resp_wave', 'Resp_time_diff_sec', 'SpO2_numeric', 'SpO2_numeric_time_diff_sec', 'Pulse_numeric', 'Pulse_numeric_time_diff_sec', 'ST_numeric', 'ST_numeric_time_diff_sec', 'Tskin_numeric', 'Tskin_numeric_time_diff_sec', 'ABP_numeric', 'ABP_numeric_time_diff_sec', 'NBP_numeric', 'NBP_numeric_time_diff_sec', 'Perf_numeric', 'Perf_numeric_time_diff_sec', 'PPV_numeric', 'PPV_numeric_time_diff_sec', 'AlertRow', 'ClosestNursing', 'Nursing_TimeDiffSec'])

In [ ]:
DataConverter = {
    "admission_periods": get_admission_periods(patient_id),
    "alarms": {},
    "nursing_records": get_nursing_record(patient_id),
    "waveforms": {}
}